# Writing a Naive Bayes Spelling Correcter

### Start by defining the word dictionary and writing the spelling correcter function

In [ ]:
import time
import string

# RegEx library
import re

# Counter module allows you to count instances of a value appears
from collections import Counter

# Example use of Counter container
c1 = Counter('abcabcab')
print("Initial:", c1)
c1.update('c')
print("Updated:", c1)

# More complex use of Counter using words
c2 = Counter(['hi', 'bye', 'bye'])
print("Word counter:", c2)

In [ ]:
# Store the words in a counter to count up the occurrences
WORDS = Counter(words(open('big.txt').read()))
SUM = sum(WORDS.values())

def words(text):
    
    '''
    Takes as input some text and returns all words in lowercase.
    '''
    
    # Regex expression for all words with one or more characters
    regex = r'\w+'
    
    # Returns a list of lowercase strings that match the regex expression 
    return re.findall(regex, text.lower())

def P(word, total=SUM):
    return WORDS[word]/total

def correction(word, num_edits=2):
    
    '''
    Return the highest probabiliy spelling correction based on word occurences
    '''
    
    return max(candidates(word, num_edits), key=P)

def candidates(word, num_edits=2):
    
    '''
    Return one or more of possible spelling corrections of the word within the known words
    and the edited words. Order or priority is as follows:
    1. known([word]) - The word if it is known, OR;
    2. known(edits1(word)) - One edit distance corrections if any of the corrections are known, OR;
    3. known(multiple_edits(word)) - Multiple edit distance corrections if any of the corrections are known, OR;
    4. [word] - The word itself, even though it is not known
    ''' 
    
    return (known([word]) or known(edits1(word)) or known(multiple_edits(word, num_edits=num_edits)) or [word])

def known(words): 
    
    '''
    The subset of words that appear in the known words.
    '''
    
    return set(w for w in words if w in WORDS)

def edits1(word):
    
    '''
    All edits that are one edit away from the input word:
     - Delete one letter
     - Transpose/swap adjacent letters
     - Replace one letter for another letter in the alphabet
     - Add a letter in the alphabet
    '''
    
    # Split the word in every possible combination (e.g., cat = [('','cat'), ('c','at'), ('ca','t'), ('cat','')]
    splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    
    # Delete the first letter of the right split for every split pair and add back to left split
    deletes    = [L + R[1:] for L, R in splits if R]
    
    # Swap the first and second letter of the right split for every split pair and add back to left split
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    
    # Replace the first letter of the right split with each letter in the alphabet for every split pair
    replaces   = [L + c + R[1:] for L, R in splits if R for c in string.ascii_lowercase]
    
    # Insert each letter in the alphabet between the left and right split for every split pair
    inserts    = [L + c + R for L, R in splits for c in string.ascii_lowercase]
    
    # Return the set of all possible editted words
    return set(deletes + transposes + replaces + inserts)

def multiple_edits(word, num_edits=2): 
    '''
    All edits that are num_edits edits away from the input word by editting the words multiple times
    '''
    
    # Create a list of words to make edits to, and a temporary set to hold each level of edits
    # Also declare a counter for the number of edits
    word_list = [word]
    temp_set = set()
    edits_count = 0
    
    # Loop until the edits_count reaches the num_edits desired 
    while edits_count < num_edits:
        
        # For each word in the word_list, make the edits to the word and append it to the temporary set
        for word in word_list:
            for e1 in edits1(word):
                temp_set.add(e1)  
         
        # Copy the temporary set to the word_list and clear the temporary set. Increment the edits_count
        word_list = temp_set.copy()
        temp_set.clear()
        edits_count+=1
    
    return set(word_list)

### Run some basic checks on the functions

In [ ]:
# Test the multiple_edits
found_1 = multiple_edits('cat', num_edits=1)
found_2 = multiple_edits('cat', num_edits=2)
start = time.time()
found_3 = multiple_edits('cat', num_edits=3)
finish = time.time()
print("Number of words found:", len(found_3))
print("Time taken in seconds:", finish-start)

# Check if original word is in the editted words list
if 'cat' in found_1:
    print("True")

# Check if lower level edits can be found in the higher level edit list
for edit in found_1:
    if edit not in found_2:
        print("Not found")
        break

In [ ]:
# With increased edit distance, accuracy can vary
for dist in [2,3]:
    print(f"Edit distance {dist}: {correction('speling', dist)}")

In [ ]:
# Alternatively
for dist in [2,3]:
    print(f"Edit distance {dist}: {correction('korrectud', dist)}")

In [ ]:
# Adding another mispelled letter
for dist in [2,3]:
    print(f"Edit distance {dist}: {correction('korrectudd', dist)}")

In [ ]:
for dist in [2,3]:
    print(f"Edit distance {dist}: {correction('ttty', dist)}")

### Some words that were confused without edit distance = 3

In [ ]:
ed3_text = """purple perpul
curtains courtens
minutes muinets
successful sucssuful
hierarchy heiarky
profession preffeson
weighted wagted
inefficient ineffiect
availability avaiblity
thermawear thermawhere
nature natior
dissension desention
unnecessarily unessasarily
disappointing dissapoiting
acquaintances aquantences
thoughts thorts
criticism citisum
immediately imidatly
necessary necasery
necessary nessasary
necessary nessisary
unnecessary unessessay
night nite
minutes muiuets
assessing accesing
necessitates nessisitates
"""

def parse_ed3(text):
    
    correct_list = []
    mispelled_list = []
    
    for line in text.split('\n'):
        if len(tuple(line.split(' '))) != 2:
            continue
        correct, mispelled = tuple(line.split(' '))
        correct_list.append(correct)
        mispelled_list.append(mispelled)
        
    return correct_list, mispelled_list

right, wrong = parse_ed3(ed3_text)
for r,w in zip(right,wrong):
    if correction(w, 3) == r:
        print(f"Corrected {w} to {correction(w, 3)}!")
    else:
        print(f"Miscorrected {w} to {correction(w, 3)} :(")